# Generalizing a Taylor Recurrence

In [2]:
from sumpy.recurrence import _make_sympy_vec, get_processed_and_shifted_recurrence

from sumpy.expansion.diff_op import (
    laplacian,
    make_identity_diff_op,
)

from sumpy.recurrence import get_recurrence, recurrence_from_pde, shift_recurrence, get_shifted_recurrence_exp_from_pde, _extract_idx_terms_from_recurrence, process_recurrence_relation, get_taylor_recurrence

import sympy as sp
from sympy import hankel1

import numpy as np

import math

import matplotlib.pyplot as plt
from matplotlib import cm, ticker

In [3]:
var = _make_sympy_vec("x", 2)
s = sp.Function("s")
n = sp.symbols("n")
i = sp.symbols("i")

In [4]:
def compute_derivatives(p):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    g_x_y = sp.log(sp.sqrt((var[0]-var_t[0])**2 + (var[1]-var_t[1])**2))
    derivs = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                        for i in range(p)]
    return derivs

In [5]:
def compute_derivatives_h2d(p, k=1.0):
    var = _make_sympy_vec("x", 2)
    var_t = _make_sympy_vec("t", 2)
    abs_dist = sp.sqrt((var[0]-var_t[0])**2 +
                        (var[1]-var_t[1])**2)
    g_x_y = (1j/4) * hankel1(0, k * abs_dist)
    derivs_helmholtz = [sp.diff(g_x_y,
                        var_t[0], i).subs(var_t[0], 0).subs(var_t[1], 0)
                                                for i in range(p)]
    return derivs_helmholtz

In [6]:
w = make_identity_diff_op(2)
laplace2d = laplacian(w)

w = make_identity_diff_op(2)
helmholtz2d = laplacian(w) + w

derivs_lap = compute_derivatives(8)
derivs_helm = compute_derivatives_h2d(8)

In [10]:
t_recurrence = get_taylor_recurrence(laplace2d)[1]
t_recurrence

(-5*n + (n + 1)**2 + 1)*s(n - 2)/x1**2

In [11]:
coord_dict = {var[0]: np.random.rand(), var[1]: np.random.rand()}

In [22]:
def eval_taylor_recurrence(deriv_order, taylor_order=4):
    terms_to_compute = taylor_order + deriv_order
    exp = 0
    for i in range(taylor_order):
        exp += t_recurrence.subs(n, deriv_order+i)/math.factorial(i) * var[0]**i
    return exp

In [26]:
eval_taylor_recurrence(3)

10*x0**3*s(4)/(3*x1**2) + 6*x0**2*s(3)/x1**2 + 6*x0*s(2)/x1**2 + 2*s(1)/x1**2

In [ ]:
eval_taylor_recurrence(n).subs(s(n+1), )

x0**3*(-5*n + (n + 4)**2 - 14)*s(n + 1)/(6*x1**2) + x0**2*(-5*n + (n + 3)**2 - 9)*s(n)/(2*x1**2) + x0*(-5*n + (n + 2)**2 - 4)*s(n - 1)/x1**2 + (-5*n + (n + 1)**2 + 1)*s(n - 2)/x1**2

In [31]:
eval_taylor_recurrence(n).subs(s(n+1), t_recurrence.subs(n,n+1))

x0**3*(-5*n + (n + 2)**2 - 4)*(-5*n + (n + 4)**2 - 14)*s(n - 1)/(6*x1**4) + x0**2*(-5*n + (n + 3)**2 - 9)*s(n)/(2*x1**2) + x0*(-5*n + (n + 2)**2 - 4)*s(n - 1)/x1**2 + (-5*n + (n + 1)**2 + 1)*s(n - 2)/x1**2

In [43]:
derivs_lap[0].subs(var[0],0)

log(sqrt(x1**2))

In [42]:
derivs_lap[1].subs(var[0],0)

0

In [44]:
derivs_lap[2].subs(var[0],0)

x1**(-2)

In [45]:
t_recurrence.subs(n, 2)

0